In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pymysql


# Function to connect to MySQL database
def connect_db():
    try:
        connection = pymysql.connect(
            host="localhost",
            user="root",
            passwd="akash123",
            database="hospitalmanagement"
        )
        return connection
    except pymysql.MySQLError as err:
        messagebox.showerror("Database Error", f"Error: {err}")
        return None


# Function to insert data into the database
def add_record():
    connection = connect_db()
    if not connection:
        return

    try:
        cursor = connection.cursor()
        query = """
            INSERT INTO patients (NameOfTablet, City, Dose, LotSize, IssueDate, ExpiryDate, 
                                  ReferenceNo, DailyDoseQuantity, SideEffect, FurtherInfo,
                                  BloodPressure, Storage, Medicine, PatientID, RGHSNo, 
                                  PatientName, DOB, NextDueDate) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = (
            name_of_tablet.get(), city.get(), dose.get(), lot_size.get(), issue_date.get(),
            expiry_date.get(), reference_no.get(), daily_dose_quantity.get(), side_effect.get(),
            further_info.get(), blood_pressure.get(), storage.get(), medicine.get(),
            patient_id.get(), rghs_no.get(), patient_name.get(), dob.get(), next_due_date.get()
        )
        cursor.execute(query, values)
        connection.commit()
        messagebox.showinfo("Success", "Record Added Successfully!")
        display_records()
        generate_prescription()
    except Exception as e:
        messagebox.showerror("Error", f"Failed to Insert Record: {e}")
    finally:
        connection.close()

# Function to update a selected record
def update_record():
    selected_item = records_tree.selection()
    if not selected_item:
        messagebox.showwarning("Warning", "Please select a record to update")
        return

    connection = connect_db()
    if not connection:
        return

    try:
        cursor = connection.cursor()
        record_id = records_tree.item(selected_item, "values")[13]  # Assuming PatientID is at index 13
        query = """
            UPDATE patients SET NameOfTablet=%s, City=%s, Dose=%s, LotSize=%s, IssueDate=%s, ExpiryDate=%s, 
                                ReferenceNo=%s, DailyDoseQuantity=%s, SideEffect=%s, FurtherInfo=%s, 
                                BloodPressure=%s, Storage=%s, Medicine=%s, RGHSNo=%s, 
                                PatientName=%s, DOB=%s, NextDueDate=%s
            WHERE PatientID=%s
        """
        values = (
            name_of_tablet.get(), city.get(), dose.get(), lot_size.get(), issue_date.get(),
            expiry_date.get(), reference_no.get(), daily_dose_quantity.get(), side_effect.get(),
            further_info.get(), blood_pressure.get(), storage.get(), medicine.get(),
            rghs_no.get(), patient_name.get(), dob.get(), next_due_date.get(), record_id
        )
        cursor.execute(query, values)
        connection.commit()
        messagebox.showinfo("Success", "Record Updated Successfully!")
        display_records()
    except Exception as e:
        messagebox.showerror("Error", f"Failed to Update Record: {e}")
    finally:
        connection.close()

# Function to delete a selected record
def delete_record():
    selected_item = records_tree.selection()
    if not selected_item:
        messagebox.showwarning("Warning", "Please select a record to delete")
        return

    connection = connect_db()
    if not connection:
        return

    try:
        cursor = connection.cursor()
        record_id = records_tree.item(selected_item, "values")[13]  # Assuming PatientID is at index 13
        query = "DELETE FROM patients WHERE PatientID=%s"
        cursor.execute(query, (record_id,))
        connection.commit()
        messagebox.showinfo("Success", "Record Deleted Successfully!")
        display_records()
    except Exception as e:
        messagebox.showerror("Error", f"Failed to Delete Record: {e}")
    finally:
        connection.close()


# Function to display records in the Treeview
def display_records():
    for record in records_tree.get_children():
        records_tree.delete(record)

    connection = connect_db()
    if not connection:
        return

    try:
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM patients")
        rows = cursor.fetchall()
        for row in rows:
            records_tree.insert("", "end", values=row)
    except Exception as e:
        messagebox.showerror("Error", f"Failed to Fetch Records: {e}")
    finally:
        connection.close()


# Function to clear all input fields
def clear_fields():
    for var in all_vars:
        var.set("")
    prescription_textbox.delete('1.0', tk.END)


# Function to generate the prescription text
def generate_prescription():
    prescription_textbox.delete('1.0', tk.END)
    prescription = (
        f"Patient Name: {patient_name.get()}\n"
        f"Patient ID: {patient_id.get()}\n"
        f"Tablet: {name_of_tablet.get()}\n"
        f"Dose: {dose.get()}\n"
        f"Daily Dose Quantity: {daily_dose_quantity.get()}\n"
        f"Issue Date: {issue_date.get()}\n"
        f"Expiry Date: {expiry_date.get()}\n"
        f"Further Information: {further_info.get()}\n"
        f"Next Due Date: {next_due_date.get()}"
    )
    prescription_textbox.insert(tk.END, prescription)


# Main window
root = tk.Tk()
root.title("Hospital Management System : Created By Akash Pandey")
root.geometry("1300x700")

tablet_options = ["Paracetamol", "Ibuprofen", "Amoxicillin", "Ciprofloxacin",
                  "Metformin", "Aspirin", "Cetirizine", "Omeprazole",
                  "Salbutamol", "Atorvastatin", "Losartan", "Ranitidine",
                  "Azithromycin", "Levothyroxine", "Diclofenac", "Clopidogrel",
                  "Pantoprazole", "Metronidazole", "Hydroxychloroquine", "Doxycycline"]

name_of_tablet = tk.StringVar()
city = tk.StringVar()
dose = tk.StringVar()
lot_size = tk.StringVar()
issue_date = tk.StringVar()
expiry_date = tk.StringVar()
reference_no = tk.StringVar()
daily_dose_quantity = tk.StringVar()
side_effect = tk.StringVar()
further_info = tk.StringVar()
blood_pressure = tk.StringVar()
storage = tk.StringVar()
medicine = tk.StringVar()
patient_id = tk.StringVar()
rghs_no = tk.StringVar()
patient_name = tk.StringVar()
dob = tk.StringVar()
next_due_date = tk.StringVar()

all_vars = [name_of_tablet, city, dose, lot_size, issue_date, expiry_date, reference_no,
            daily_dose_quantity, side_effect, further_info, blood_pressure, storage,
            medicine, patient_id, rghs_no, patient_name, dob, next_due_date]

# UI Components
title = tk.Label(root, text="HOSPITAL MANAGEMENT SYSTEM", font=("Times New Roman", 30, "bold"), fg="red")
title.pack(side="top", fill="x")

# Patient Information Frame
info_frame = tk.LabelFrame(root, text="Patient Information", bg="yellow", font=("Arial", 12, "bold"))
info_frame.place(x=10, y=50, width=900, height=320)

# Patient Information Labels and Entries
fields = [("Name Of Tablet", name_of_tablet), ("City", city), ("Dose", dose), ("Lot Size", lot_size),
          ("Issue Date", issue_date), ("Expiry Date", expiry_date), ("Reference No.", reference_no),
          ("Daily Dose Quantity", daily_dose_quantity), ("Side Effect", side_effect),
          ("Further Information", further_info), ("Blood Pressure", blood_pressure),
          ("Storage", storage), ("Medicine", medicine), ("Patient ID", patient_id),
          ("RGHS No.", rghs_no), ("Patient Name", patient_name), ("DOB", dob), ("Next Due Date", next_due_date)]

for i, (label, var) in enumerate(fields):
    tk.Label(info_frame, text=label, bg="yellow", font=("Arial", 10)).grid(row=i//2, column=(i%2)*2, padx=10, pady=5, sticky="w")
    if label == "Name Of Tablet":
        ttk.Combobox(info_frame, textvariable=var, values=tablet_options, width=28).grid(row=i//2, column=(i%2)*2+1, padx=10, pady=5)
    else:
        tk.Entry(info_frame, textvariable=var, width=30).grid(row=i//2, column=(i%2)*2+1, padx=10, pady=5)

# Table to Display Records with Scrollbar
records_frame = tk.Frame(root)
records_frame.place(x=10, y=420, width=1500, height=250)

columns = [field[0] for field in fields]
records_tree = ttk.Treeview(records_frame, columns=columns, show="headings")

#Scrollbars
scroll_x = ttk.Scrollbar(records_frame, orient="horizontal", command=records_tree.xview)
scroll_y = ttk.Scrollbar(records_frame, orient="vertical", command=records_tree.yview)
records_tree.configure(xscrollcommand=scroll_x.set, yscrollcommand=scroll_y.set)

# Headings
for col in columns:
    records_tree.heading(col, text=col)
    records_tree.column(col, width=120)

# Add the treeview to the frame
records_tree.pack(fill="both", expand=True)

scroll_x.pack(side="bottom", fill="x")
scroll_y.pack(side="right", fill="y")

# Buttons
btn_frame = tk.Frame(root)
btn_frame.place(x=10, y=370, width=1200)

buttons = [("Prescription", add_record), ("Prescription Data", display_records),
           ("Update Record",update_record), ("Delete Record",delete_record),
           ("Clear", clear_fields), ("Exit", root.quit)]

for i, (btn_text, command) in enumerate(buttons):
    tk.Button(btn_frame, text=btn_text, width=25, height=2, bg="green", fg="white", command=command).grid(row=0, column=i, padx=10)

# Prescription Text Box
prescription_frame = tk.LabelFrame(root, text="Prescription", font=("Arial", 12, "bold"))
prescription_frame.place(x=920, y=50, width=400, height=320)
prescription_textbox = tk.Text(prescription_frame, font=("Arial", 10), wrap="word",bg="powder blue")
prescription_textbox.pack(fill="both", expand=True)

# Run the application
root.mainloop()